In [12]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit import Parameter
from scipy.optimize import minimize
import time

In [13]:
def create_hamiltonian():
    """Creates the Hamiltonian operator using SparsePauliOp."""
    # H = -0.81 * II + 0.17 * ZZ + 0.045 * XX - 0.045 * YY (Original H2 example)
    pauli_list = [("II", -0.81), ("ZZ", 0.17), ("XX", 0.045), ("YY", -0.045)]

    hamiltonian = SparsePauliOp.from_list(pauli_list)
    print(f"Hamiltonian:\n{hamiltonian}")
    
    return hamiltonian

In [14]:
num_params = 5 
theta = [Parameter(f'θ{i}') for i in range(num_params)]

In [15]:
def uccsd_ansatz():
    """
    Implements a simplified UCCSD-like ansatz circuit for 2 qubits
    using Parameter objects.
    Returns:
        QuantumCircuit: The parameterized ansatz circuit template.
    """
    num_qubits = 2
    circuit = QuantumCircuit(num_qubits)

    circuit.rx(theta[0], 0)
    circuit.rz(theta[1], 0)
    circuit.rx(theta[2], 1)
    circuit.rz(theta[3], 1)
    circuit.cx(0, 1)
    circuit.ry(theta[4], 1)
    circuit.cx(0, 1)
    circuit.barrier()

    print(f"Ansatz created with {circuit.num_parameters} parameters.") 
    return circuit

In [16]:
iteration_count = 0
ansatz_template = uccsd_ansatz()

Ansatz created with 5 parameters.


In [17]:
def calculate_expectation(params, hamiltonian, estimator, callback_dict):
    """
    Calculates the expectation value using the Estimator primitive.
    Args:
        params (np.array): Current parameters from the optimizer.
        hamiltonian (SparsePauliOp): The Hamiltonian operator.
        estimator (BaseEstimator): The Qiskit Estimator primitive.
        callback_dict (dict): Dictionary to store intermediate results for tracking.
    Returns:
        float: The calculated expectation value (energy).
    """
    global iteration_count
    iteration_count += 1

    job = estimator.run(
        circuits=[ansatz_template],    
        observables=[hamiltonian],
        parameter_values=[params]      
    )
    result = job.result() 

    expectation_value = result.values[0]

    callback_dict['params'].append(params)
    callback_dict['energy'].append(expectation_value)
    if iteration_count % 10 == 0: 
       print(f"Iteration: {iteration_count}, Energy: {expectation_value:.6f}, Params: {np.round(params, 3)}")

    return expectation_value

In [18]:
def vqe_optimization(hamiltonian, estimator):
    """
    Runs the VQE optimization loop.
    Args:
        hamiltonian (SparsePauliOp): The Hamiltonian.
        estimator (BaseEstimator): The Qiskit Estimator primitive.
    Returns:
        OptimizeResult: The result object from scipy.optimize.minimize.
    """
    initial_params = np.random.rand(num_params) * 2 * np.pi

    print(f"Number of parameters in ansatz: {ansatz_template.num_parameters}") 
    print(f"Number of parameters for optimizer: {num_params}")
    assert ansatz_template.num_parameters == num_params 

    print(f"Initial parameters: {initial_params}")

    callback_dict = {'params': [], 'energy': []}

    start_time = time.time()
    result = minimize(
        fun=calculate_expectation,     
        x0=initial_params,              
        args=(hamiltonian, estimator, callback_dict),
        method="COBYLA",
        options={'maxiter': 200, 'disp': False} 
    )
    end_time = time.time()
    print(f"\nOptimization finished in {end_time - start_time:.2f} seconds.")

    result.intermediate_info = callback_dict

    return result

In [ ]:
if __name__ == "__main__":
    aer_estimator = AerEstimator(
        run_options={"shots": 1024},
        transpile_options={"optimization_level": 0}
    )

    hamiltonian_op = create_hamiltonian()

    print("\nStarting VQE Optimization...")
    vqe_result = vqe_optimization(hamiltonian_op, aer_estimator)

    print("\n--- VQE Results ---")
    print(f"Optimized parameters: {vqe_result.x}")
    print(f"Final ground state energy: {vqe_result.fun:.6f}")


Hamiltonian:
SparsePauliOp(['II', 'ZZ', 'XX', 'YY'],
              coeffs=[-0.81 +0.j,  0.17 +0.j,  0.045+0.j, -0.045+0.j])

Starting VQE Optimization...
Number of parameters in ansatz: 5
Number of parameters for optimizer: 5
Initial parameters: [3.6579906  1.35680848 3.13614341 5.84839409 4.57997446]
Iteration: 10, Energy: -0.871826, Params: [5.172 0.959 2.36  5.797 4.393]
Iteration: 20, Energy: -0.973594, Params: [5.163 0.75  1.686 4.4   4.857]
Iteration: 30, Energy: -0.974434, Params: [5.186 0.779 1.636 4.624 5.007]
Iteration: 40, Energy: -0.972500, Params: [5.068 0.82  1.577 4.587 5.013]
Iteration: 50, Energy: -0.976436, Params: [5.057 0.813 1.584 4.586 5.008]
Iteration: 60, Energy: -0.969639, Params: [5.058 0.812 1.584 4.586 5.007]
Iteration: 70, Energy: -0.973896, Params: [5.058 0.812 1.584 4.586 5.007]

Optimization finished in 1.60 seconds.

--- VQE Results ---
Optimized parameters: [5.05797541 0.81218177 1.58357115 4.58640547 5.00655218]
Final ground state energy: -0.973896


In [20]:
pauli_list = [("II", -0.81), ("ZZ", 0.17), ("XX", 0.045), ("YY", -0.045)]
hamiltonian_op = SparsePauliOp.from_list(pauli_list)
matrix = hamiltonian_op.to_matrix()
eigenvalues, _ = np.linalg.eigh(matrix) 
exact_ground_state_energy = np.min(eigenvalues)
print(f"Exact Ground State Energy: {exact_ground_state_energy.real:.6f}")

Exact Ground State Energy: -0.980000
